In [6]:
import pandas as pd
from deep_translator import (GoogleTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator,
                             YandexTranslator,
                             single_detection,
                             batch_detection)
import nltk
import string as st
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentAnalyzer
import CleanupFunctions as clean
import AddedWords

# Haalt info uit csv op.
dfDutch = pd.read_csv("CSVFiles/Mock-updata.csv", sep=';')

dfSentiment = pd.DataFrame.from_dict({
    "id":[],
    "text": [],
    "compound": [],
    "neg":[],
    "neu":[],
    "pos":[],
})

stopwords = nltk.corpus.stopwords.words("english")

index = 0

sia = SentimentIntensityAnalyzer()
# Update de woordenlijst, met zelftoegevoegde woorden en het sentiment, bron: https://ithaka.github.io/tdm-notebooks/sentiment-analysis-with-vader.html
sia.lexicon.update(AddedWords.added_words())

#Loop die er voor zorgt dat alle vertaalde reviews wordt uitgeprint
for col_name, review in dfDutch.iterrows():
    translated = GoogleTranslator(source='dutch', target='english').translate(text=dfDutch.loc[col_name]['Tekst'])
    textlowercase = clean.convert_to_lowercase(translated)
    textclean = clean.punc_clean(textlowercase)
    textstopword = clean.remove_stopword(textclean)
    tokenized = sent_tokenize(textstopword)
    for line in tokenized:     
        ss = sia.polarity_scores(line)
        dfTemp = pd.DataFrame.from_dict({
                            "id":[col_name],
                            "text": [textstopword],
                            "compound": [ss["compound"]],
                            "neg":[ss["neg"]],
                            "neu":[ss["neu"]],
                            "pos":[ss["pos"]],
                            })
        index = index + 1
    dfSentiment = pd.concat([dfSentiment, dfTemp], ignore_index=True)

print(dfSentiment)
dfSentiment.to_csv("CSVFiles/MockUpDataSentiment.csv")



      id                                               text  compound    neg  \
0    0.0      totally disagree wait surgery bloody hospital   -0.8431  0.751   
1    1.0  receptionist rude makes wait phone waiting 50 ...   -0.8591  0.393   
2    2.0                              great sister handsome    0.8074  0.000   
3    3.0  satisfied corona booster risk group go back sc...    0.8901  0.078   
4    4.0  unfortunately lung cancer chance surviving sis...   -0.0258  0.303   
5    5.0  broke ribs hurts lot sit emergency room waitin...   -0.8595  0.491   
6    6.0  give birth hospital teenage mother midwives to...   -0.1779  0.201   
7    7.0  recorded say corona total bullshit not dead dr...   -0.5938  0.321   
8    8.0  blood tests results yet wait know always short...   -0.1882  0.185   
9    9.0  top hospital pay 1006 euros treatment not work...    0.0000  0.264   
10  10.0  care poor waiting time simple surgery knee two...   -0.4404  0.399   
11  11.0      receptionist curt wait lon